In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

## Import data

In [63]:
data_repo = "../data/"
IMPORT_PARAMS = {'sep': '|', 'encoding': 'latin-1'}

In [130]:
def import_contract_history(file):
    date_cols = ['CRE_DATE', 'UPD_DATE', 'DATE_RESILIATION', 'DATE_DEBUT', 'DATE_FIN']
    contract_history = pd.read_csv(file, dtype={'NUM_CAMPAGNE': object, 'INSTANCE_ID': object}, parse_dates=date_cols, **IMPORT_PARAMS)

    
    for c in ['CRE_DATE', 'UPD_DATE']:
        contract_history['{}_date'.format(c)] = pd.to_datetime(contract_history[c].map(lambda x: datetime.datetime.date(x)))
    
    contract_history['NUM_CAMPAGNE'] = contract_history['NUM_CAMPAGNE'].map(lambda x: x if x not in ['N', ''] else "NAN")
    contract_history.OPTION.fillna("NO", inplace=True)
    
    useless_variables = [ 'CONTRACT_NUMBER', 'CONTRACT_MODIFICATEUR', 'CRE_DATE', 
                       'CONDITION_REGLEMENT', 'MOTIF_RESILIATION', 'RENOUVELLEMENT_AGENCE',
                       'PRIX_FORMULE', 'PRIX_OPTION', 'NUM_CAMPAGNE', 'DATE_RESILIATION']
    
    contract_history.drop(useless_variables, axis=1, inplace=True)
    
    return contract_history

def import_equipment(file):
    equipment = pd.read_csv(file, dtype={'CODE_POSTAL': object, 'INSTANCE_ID': object, 'ORGANISATION_ID': object}, parse_dates=['INSTALL_DATE', 'RACHAT_DATE'], **IMPORT_PARAMS)
    equipment['POINTS_FIDEL'].fillna(0, inplace=True)
    equipment['TYPE_VOIE'].fillna('RUE', inplace=True)
    equipment['STOP_PHONING'].fillna('N', inplace=True)
    
    # keep only rachat_code which appears more than 1000 times in the base. replace other by "AUTRE"
    value_count = equipment.RACHAT_CODE.value_counts()
    code_list = value_count.index[value_count.lt(1000)]
    equipment.RACHAT_CODE.replace(code_list, "AUTRE")
    equipment.RACHAT_CODE.fillna("NA", inplace=True)
    
    equipment.drop(['RACHAT_LIB', 'ESCALIER', 'NUMERO', 'ETAGE', 'RUE', 'VILLE', 'RACHAT_DATE', 'LOCALISATION_ORGANISME', 'COMPLEMENT_RUE', 'LOCATION_ID', 'PARTY_ID_OCC', 'MODELE_LIB', 'MARQUE_CODE'], axis=1, inplace=True)
    return equipment

def import_organisation(file):
    organisation = pd.read_csv(file, dtype={'L2_ORGANISATION_ID': object, 'L2_ORGA_CODE_POSTAL': object, 'L1_ORGANISATION_ID': object, 'CIA': object}, **IMPORT_PARAMS)
    useless_variables = ['L1_NAME', 'L2_NAME', 'ADRESSE']
    organisation.drop(useless_variables, axis=1, inplace=True)
    return organisation

def preproc_intervention(data):
    data.NB_PASSAGE.fillna(1, inplace=True)
    data.PROBLEM_CODE.fillna('NC', inplace=True)
    data.ORIGINE_INCIDENT.fillna('NAN', inplace=True)
    
    origines = ['Téléphone', 'Automatique', 'Mail', 'Courrier', 'Répondeur','Venue client', 'NAN', 'Internet', 'Télé-alarme']
    #'FAX' modality appears in test set but not in train
    data.loc[~data["ORIGINE_INCIDENT"].isin(origines), "ORIGINE_INCIDENT"] = "NAN"
    
    data.COMMENTAIRE_BI.fillna('.', inplace=True)
    data.RESOURCE_ID.fillna(0, inplace=True)
    
    missing_millesime = data.MILLESIME.isnull()
    data.loc[missing_millesime, 'MILLESIME'] = data.loc[missing_millesime, 'CRE_DATE_GZL'].apply(lambda x: x.year)
    
    data.drop(['PROBLEM_DESC', 'SS_TR_FLAG'], axis=1, inplace=True)
    data.drop_duplicates(inplace=True)
    
    data['MILLESIME'] = data['MILLESIME'].astype(str)
    data['RESOURCE_ID'] = data['RESOURCE_ID'].astype(str)

def merge_data(data_, 
               equipement_df, 
               orga_df, 
               contrat_history_df, 
               nature_code_eau_chaude, 
               nature_code_energie, 
               nature_code_fonction, 
               nature_code_installation, 
               nature_code_specification):
    
    data = data_.merge(equipement_df, how='left', on='INSTANCE_ID').merge(orga_df, how='left', left_on='ORGANISATION_ID', right_on='L2_ORGANISATION_ID')
    contrat_history_s = data[['INCIDENT_NUMBER', 'INSTANCE_ID','CRE_DATE_GZL']].merge(contrat_history_df).query('CRE_DATE_GZL>=UPD_DATE_date')
    contrat_history_s = contrat_history_s.sort_values(['INCIDENT_NUMBER', 'UPD_DATE'], ascending=[True,False]).drop_duplicates(keep='first', subset=['INCIDENT_NUMBER'])
    data = data.merge(contrat_history_s, how='left').merge(nature_code_eau_chaude, how='left')
    data = data.merge(nature_code_energie, how='left').merge(nature_code_fonction, how='left')
    data = data.merge(nature_code_installation, how='left').merge(nature_code_specification, how='left')
    
    drop_columns = ['CRE_DATE_date', 'UPD_DATE_date']
    data.drop(drop_columns, axis=1, inplace=True)
    
    print(data.shape[0], data.INCIDENT_NUMBER.nunique(), data.INSTANCE_ID.nunique())
    return data

def import_all(file, history=False):
    contract_histo = import_contract_history(data_repo + 'contract_history.csv')
    equipment = import_equipment(data_repo + 'equipment.csv')
    organisation = import_organisation(data_repo + 'organisation.csv')
    
    code_eau_chaude = pd.read_csv(data_repo + 'nature_code_eau_chaude.csv', sep='|')
    code_energie = pd.read_csv(data_repo + 'nature_code_energie.csv', sep='|')
    code_fonction = pd.read_csv(data_repo + 'nature_code_fonction.csv', sep='|')
    code_installation = pd.read_csv(data_repo + 'nature_code_installation.csv', sep='|')
    code_specification = pd.read_csv(data_repo + 'nature_code_specification.csv', sep='|')
    
    date_cols = ['CRE_DATE_GZL', 'SCHEDULED_START_DATE', 'SCHEDULED_END_DATE']
    
    if history: 
        merged_list = []
        chunksize = 10 ** 6
        date_cols += ['ACTUAL_START_DATE', 'ACTUAL_END_DATE']
        
        for chunk in pd.read_csv(file, 
                                 sep='|', 
                                 encoding='latin-1', 
                                 dtype={  'INSTANCE_ID': object, 
                                            'INCIDENT_NUMBER': object, 
                                            'INCIDENT_TYPE_ID': object, 
                                            'NUM_CAMPAGNE': object,
                                            'MILLESIME': object,
                                            'RESOURCE_ID': object,
                                            'AUTEUR_INCIDENT': object}, 
                                 parse_dates=date_cols, 
                                 chunksize=chunksize):
            preproc_intervention(chunk)
        
            merged_list.append(merge_data(chunk, 
                                          equipment, 
                                          organisation, 
                                          contract_histo,
                                          code_eau_chaude,
                                          code_energie,
                                          code_fonction,
                                          code_installation,
                                          code_specification))
    
        return pd.concat(merged_list)
    
    else:    
        intervention = pd.read_csv(file, 
                                   sep='|', 
                                   encoding='latin-1', 
                                   dtype={'INSTANCE_ID': object, 
                                        'INCIDENT_NUMBER': object, 
                                        'INCIDENT_TYPE_ID': object, 
                                        'NUM_CAMPAGNE': object,
                                        'MILLESIME': object,
                                        'RESOURCE_ID': object,
                                        'AUTEUR_INCIDENT': object},
                                   parse_dates=date_cols)
        
        preproc_intervention(intervention)

        return merge_data(intervention, 
                          equipment, 
                          organisation, 
                          contract_histo,
                          code_eau_chaude,
                          code_energie,
                          code_fonction,
                          code_installation,
                          code_specification)

In [131]:
# Need to be improved to be usable
# warning: delete created variables before using

def create_target_for_history(history):
    history.sort_values(['INSTANCE_ID', 'CRE_DATE_GZL', 'INCIDENT_NUMBER'], inplace=True)
    groups = history.groupby('INSTANCE_ID')
    
    get_time_delta = lambda s: s.diff(1).shift(-1)
    history['time_delta'] = groups.ACTUAL_START_DATE.transform(get_time_delta)
    history['next_incident_type'] = groups.INCIDENT_TYPE_NAME.transform(lambda s: s.shift(-1))
    history['target_test'] = history['time_delta'].between(pd.Timedelta('1 days'), pd.Timedelta('182 days')) \
                                        & (history['next_incident_type'] != 'Entretien') \
                                        & history['MOTIF_ANNULATION_CODE'].isnull()
            
    return history

def add_train_target_value_in_history(history, train):
    history_copy = history.copy()
    train_copy = train.copy()
    
    #train_copy.reset_index(inplace=True)
    train_copy.set_index(['INSTANCE_ID', 'CRE_DATE_GZL', 'INCIDENT_NUMBER'], inplace=True)

    #history_copy.reset_index(inplace=True)
    history_copy.set_index(['INSTANCE_ID', 'CRE_DATE_GZL', 'INCIDENT_NUMBER'], inplace=True)

    train_copy.sort_index(inplace=True)
    history_copy.sort_index(inplace=True)

    history_copy = history_copy.join(train_copy['target'], how='left')
    history_copy.reset_index(inplace=True)
    return history_copy

In [132]:
train = import_all(data_repo + 'intervention_train.csv')

1048030 1048030 462663


In [133]:
#train.to_csv('../data/merged_data/train.csv')
train.to_pickle('../data/merged_data/train.pkl')

In [134]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048030 entries, 0 to 1048029
Data columns (total 56 columns):
INSTANCE_ID             1048030 non-null object
INCIDENT_NUMBER         1048030 non-null object
INCIDENT_TYPE_ID        1048030 non-null object
INCIDENT_TYPE_NAME      1048030 non-null object
TYPE_BI                 1048030 non-null object
NB_PASSAGE              1048030 non-null float64
MILLESIME               1048030 non-null object
PROBLEM_CODE            1048030 non-null object
AUTEUR_INCIDENT         1048030 non-null object
ORIGINE_INCIDENT        1048030 non-null object
COMMENTAIRE_BI          1048030 non-null object
TYPE_UT                 1048030 non-null object
GRAVITE                 1048030 non-null object
RESOURCE_ID             1048030 non-null object
SCHEDULED_START_DATE    1048030 non-null datetime64[ns]
SCHEDULED_END_DATE      1048030 non-null datetime64[ns]
CRE_DATE_GZL            1048030 non-null datetime64[ns]
target                  1048030 non-null int64

In [135]:
test = import_all(data_repo + 'intervention_test.csv')

226770 226770 226770


In [136]:
#test.to_csv('../data/merged_data/test.csv')
test.to_pickle('../data/merged_data/test.pkl')

In [11]:
#Memory error, try to improve code or do it in several slices
history = import_all(data_repo + 'intervention_history.csv', history=True)

1000000 1000000 526538
1000000 1000000 526139
1000000 1000000 526484
1000000 1000000 526741
1000000 1000000 526670
171308 171308 150181


In [12]:
history.to_csv('../data/merged_data/history.csv')

## Recreate targetby training value when possible and computed value otherwize

In [13]:
#history_extract = history.iloc[:100000]
#history_extract = create_target_for_history(history_extract)
#history_extract = add_train_target_value_in_history(history_extract, train)

In [20]:
history = create_target_for_history(history)

In [19]:
history = add_train_target_value_in_history(history, train)

In [22]:
history.head()

,INSTANCE_ID,CRE_DATE_GZL,INCIDENT_NUMBER,DATE_SAISIE_RETOUR,INCIDENT_TYPE_ID,INCIDENT_TYPE_NAME,INCIDENT_STATUS_ID,INCIDENT_STATUS_NAME,INCIDENT_STATUS_CODE,INCIDENT_CLOSED_FLAG,...,UPD_DATE_date,EAU_CHAUDE,ENERGIE,FONCTION,INSTALLATION,SPECIFICATION,target,time_delta,next_incident_type,target_test
0,10003,2014-03-10,50838796,2014-04-11,10005,Entretien,2,1-Réalisé,CLOSED,Y,...,NaT,-,FIOUL,CHAUDIERE SOL MECANIQUE,-,TIRAGE NATUREL,NaN,212 days,Dépannage,False
1,10003,2014-11-05,57407214,2014-11-14,10003,Dépannage,51,3-A revoir,WAITING,Y,...,NaT,-,FIOUL,CHAUDIERE SOL MECANIQUE,-,TIRAGE NATUREL,NaN,213 days,Entretien,False
2,10003,2015-04-22,62597704,2015-06-16,10005,Entretien,2,1-Réalisé,CLOSED,Y,...,NaT,-,FIOUL,CHAUDIERE SOL MECANIQUE,-,TIRAGE NATUREL,NaN,270 days,Dépannage+Entretien,False
3,10003,2016-03-03,71920326,2016-03-07,10004,Dépannage+Entretien,110,2-Absent,NaN,Y,...,NaT,-,FIOUL,CHAUDIERE SOL MECANIQUE,-,TIRAGE NATUREL,NaN,0 days,Entretien,False
4,10003,2016-03-04,71950200,2016-03-07,10005,Entretien,2,1-Réalisé,CLOSED,Y,...,NaT,-,FIOUL,CHAUDIERE SOL MECANIQUE,-,TIRAGE NATUREL,NaN,NaT,Entretien,False


In [24]:
target_vs_created = history.loc[~history.target.isnull(), ['target', 'target_test']]

In [25]:
from sklearn.metrics import confusion_matrix

In [26]:
confusion_matrix(target_vs_created['target'], target_vs_created['target_test'])

array([[765326,  22990],
       [ 97832, 161882]])

Target test is not perfect but good enough for the moment.<br>
No need to spend more time on it as long as I am not sure I can use history (i.e. do other tables only have obs from training or the full data ?)

If improvment is needed: 
- delete last obs of each instance_id
- check for better handling of cancelled obs (mb take them out before computing time delta)

### Equipment

In [61]:
contract = import_contract_history('../data/contract_history.csv')

In [62]:
contract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4721096 entries, 0 to 4721095
Data columns (total 11 columns):
STS_CODE         object
UPD_DATE         datetime64[ns]
DATE_DEBUT       datetime64[ns]
DATE_FIN         datetime64[ns]
INSTANCE_ID      int64
FORMULE          object
OPTION           object
CONTRAT_TARIF    int64
PRIX_FACTURE     int64
CRE_DATE_date    datetime64[ns]
UPD_DATE_date    datetime64[ns]
dtypes: datetime64[ns](5), int64(3), object(3)
memory usage: 396.2+ MB


In [139]:
#etage quality is not usable
train.ORIGINE_INCIDENT.value_counts()

Téléphone       673242
Automatique     372626
Répondeur          851
Courrier           466
Mail               263
Venue client       262
NAN                253
Internet            57
Télé-alarme         10
Name: ORIGINE_INCIDENT, dtype: int64

In [140]:
train.ORIGINE_INCIDENT.unique()

array(['Téléphone', 'Automatique', 'Mail', 'Courrier', 'Répondeur',
       'Venue client', 'NAN', 'Internet', 'Télé-alarme'], dtype=object)

In [ ]:
origines = ['Téléphone', 'Automatique', 'Mail', 'Courrier', 'Répondeur','Venue client', 'NAN', 'Internet', 'Télé-alarme']
df.loc[~df["ORIGINE_INCIDENT"].isin(origines), "ORIGINE_INCIDENT"] = "NAN"

In [142]:
train. = 'NAN' if train.ORIGINE_INCIDENT not in 

SyntaxError: invalid syntax (<ipython-input-142-52551974bea0>, line 1)

# More preproc to perform

equipment: all codes_* to review


In [ ]:
# feature ingeneering on contracts:
# dates: compute timedelta between update_date and creation_date
# computation timedelta between creation_date and incident_creation_date

In [137]:
# group modalities in FORMULE
train.FORMULE.value_counts()

SECURITE*                    744162
INTEGRAL***                  238742
SERENITE                      38567
ESSENTIEL P2                  22400
MULTISERVICES                  2675
Sécurité Pack Chauffage        1231
SECURITE* 2V                    172
SECURITE* CLIENT                 53
ESSENTIEL CLIENT                 11
Sécurité Pack Chaudière           6
TRANQUILLITE PRO PAC CLIM         6
TRANQUILLITE PRO GAZ 6/7          4
Sécurité Pack Adou Client         1
Name: FORMULE, dtype: int64

In [94]:
# complete rework of option (extract real options if possible)
train.OPTION.value_counts()

NO                                                                                                   930053
7/7                                                                                                   41586
RAMONAGE GAZ                                                                                          19835
RAMONAGE FIOUL                                                                                        17742
Garantie G5/5                                                                                         12379
Gicleur fioul / RAMONAGE FIOUL                                                                         5741
RAMONAGE GAZ / 7/7                                                                                     4219
Gicleur fioul                                                                                          3868
Solaire Sécurité                                                                                       2133
MULTISERVICES Ass. plomberie

In [102]:
# create a other modality for rare codes
train.MODELE_CODE.value_counts()

SDUCW73TN00                  10086
CMA6781031                    9027
SDU0010007670                 8823
SDUZZTHEMAC25E                8374
CMANECTRA223CF                8069
SDUDM73TN00                   7936
SDUZZTHEMA C 23/24            7194
SDUZZTHEMAPLUS C25/30E/EV     6773
CMA200006762037               6585
SDUCX73TN00                   6086
SDUVF73TN00                   5739
SDUIR52TN00                   5501
SDUTHEMIS23                   5497
SDUZZTHEMAPLUS F25E/F30E      5202
CMACALYDRA23CF                4932
SDUTHEMAC23E                  4918
CMA6787031                    4792
ELM7716705002                 4717
CMA6781037                    4717
CMA3650193                    4625
SDUZZTHEMA F 23/24 E          4534
SDUQB44TN00                   4522
SDU223C                       4382
SDUCK70TN00                   4215
SDUDN73TN00                   4189
SDUZZTHEMA F25 E/ F30 E       4128
CMA223731                     3686
SDU0010007671                 3684
CMANIAGARA28FFNAT   

In [104]:
# Create a modality other for brands with low occurance
train.MARQUE_LIB.value_counts()

SAUNIER DUVAL           247116
CHAFFOTEAUX ET MAURY    208610
ELM LEBLANC             122434
DE DIETRICH              74107
VIESSMANN                59747
FRISQUET                 51937
CHAPPEE                  49607
VAILLANT                 24817
FRANCO BELGE             18420
IDEAL STANDARD           16926
ATLANTIC                 14852
GEMINOX                  14051
FERROLI                  11941
DEVILLE                   9157
BUDERUS                   7268
DAIKIN                    7197
RIELLO                    6975
REPRISE                   6583
AUER                      6082
UNICAL                    5963
CHAUFFAGE FRANCAIS        5097
MTS                       4450
ACV                       4000
MITSUBISHI                3950
TECHNIBEL                 3466
OERTLI                    3392
STYX                      3208
HITACHI                   3204
AUTRES                    2748
GUILLOT                   2102
                         ...  
SFCP                        14
PIAZZETT

In [113]:
#create a other category on auteur incident (feature engineering mb, like more than 1000 vs less ?)
train.AUTEUR_INCIDENT.value_counts()
#same on RESOURCE_ID

36927    41255
1553     10473
1903      9664
1711      7693
7637      6864
8212      6712
24876     6618
3851      6615
24612     6169
1624      6123
2175      5742
6657      5403
1740      5351
4573      5248
2402      5211
10118     5206
15162     5170
2279      5099
9821      5047
5771      5025
7624      5015
2443      5014
7897      4987
6396      4905
7895      4836
6120      4725
1738      4633
3870      4555
1709      4515
10258     4483
         ...  
40888        1
9148         1
35240        1
42925        1
26643        1
31665        1
31177        1
10188        1
27161        1
27167        1
40897        1
27168        1
10292        1
24632        1
35180        1
28014        1
36023        1
42079        1
18791        1
17487        1
24432        1
34900        1
44576        1
41877        1
40019        1
38292        1
37790        1
10120        1
35200        1
26637        1
Name: AUTEUR_INCIDENT, Length: 2088, dtype: int64